### Imports

In [1]:
import sys
#sys.path.insert(0, "/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main")
#sys.path.remove("/work/pi_wenlongzhao_umass_edu/27/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main")

print(sys.path)


['/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main', '/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python38.zip', '/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8', '/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8/lib-dynload', '', '/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8/site-packages']


In [2]:
if 'coir' in sys.modules:
    del sys.modules['coir']

In [3]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
import pandas as pd

/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### getting the corpus & queries

In [4]:
tasks = get_tasks(tasks=["cosqa"])

in tasks 
fetched data from hf
data loader init


20604it [00:00, 236756.51it/s]


In [5]:
#tasks['cosqa'][0]['d1']

In [6]:
#expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CoSQA_deepseek_exp.csv'
expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CoSQA_explanations_query_code.csv'
expl_df = pd.read_csv(expl_path)
expl_df.rename(columns={"query_id": "query-id", "corpus_id": "corpus-id"}, inplace=True)
expl_df.head()

,Unnamed: 0,query-id,doc,corpus-id,code,explanation_granite_1,explanation_granite_2,explanation_granite_3,explanation_granite_4,explanation_deepseek_1_cleaned,explanation_deepseek_2_cleaned,explanation_deepseek_3_cleaned,explanation_deepseek_4_cleaned
0,0,q1,python code to write bool value 1,d1,"def writeBoolean(self, n):\n """"""\n ...",\nThe provided code is a Python method named `...,"\n1. `def writeBoolean(self, n):` - This line ...",\nThis code snippet is a function that writes ...,\nThe code snippet provided is a method named ...,The doc string and code both write the boolean...,"Alright, I need to break down this Python code...",The code snippet writes a boolean value to a s...,The code snippet defines a method called write...
1,1,q2,"""python how to manipulate clipboard""",d2,"def paste(xsel=False):\n """"""Returns system ...",\nThe provided Python code snippet defines a f...,\n1. `def paste(xsel=False):` - This line defi...,\nThis code snippet is a function in Python th...,\nThe code snippet provided is a Python functi...,The paste function pastes data from the clipbo...,The code snippet begins by defining a function...,The code snippet is a function called paste th...,The code snippet defines a function called pas...
2,2,q3,python colored output to html,d3,"def _format_json(data, theme):\n """"""Pretty ...","\nThis Python function, `_format_json`, takes ...","\n1. `def _format_json(data, theme):` - This l...","\nThis code snippet, named `_format_json`, is ...",\n```python\nimport json\nimport sys\nimport p...,The docstring explains that the function _form...,"```pythondef _format_json(data, theme): """"""...",This code snippet is a function called _format...,To explain the code snippet in a way that can ...
3,3,q4,"python ""create directory"" using ""relative path""",d4,"def create_path(path):\n """"""Creates a absol...","\nThe provided Python function, `create_path(p...",\n1. The code snippet defines a function named...,\nThis code snippet helps you create a new dir...,\nThe provided code snippet is a Python functi...,The doc and code create an absolute path from ...,"Okay, I need to explain this Python code snipp...",This code creates a file system where any path...,The code snippet defines a function called cre...
4,4,q5,python column of an array,d5,"def _vector_or_scalar(x, type='row'):\n """"""...",\nThe provided Python code defines a function ...,\nThis code snippet defines a function called ...,\nThis code snippet is a function that convert...,\nThe code snippet provided is a function defi...,The code checks if an object `x` is an array o...,"Alright, so I've got this Python code snippet ...",The code snippet is a Python function called _...,"Okay, I'm going to try to figure out how to ex..."


In [7]:
# for col in expl_df.columns:
#     nan_rows = expl_df[expl_df[col].isna()]
#     if not nan_rows.empty:
#         print(f"Column: {col} - NaN Count: {nan_rows.shape[0]}")
#         #print(nan_rows)  # Print the rows with NaN values for this column
#         print("-" * 50)  # Separator for readability

In [8]:
# expl_df.columns

### adding explanation to df

#### multiple explanations 

In [9]:
# corpus, queries, qrels = tasks['cosqa']
# new_corpus = {}
# new_qrels = {}

# for _, row in expl_df.iterrows():
#     base_corpus_id = row['corpus-id']  # Existing base document ID (e.g., d1, d2, etc.)
#     query_id = row['query-id']         # Query ID (e.g., q1, q2, etc.)
    
#     # Iterate over multiple explanations and create new corpus entries
#     for i in range(1, 4):  # Assuming 4 explanations per document
#         explanation_col = f'explanation_deepseek_{i}_cleaned'
#         if explanation_col in row and row[explanation_col] and row[explanation_col].strip():
#             new_corpus_id = f"{base_corpus_id}_{i}"  # e.g., d1_1, d1_2, ...

#             # Store new document in corpus
#             new_corpus[new_corpus_id] = {
#                 "text": row[explanation_col],
#                 "title": ""
#             }

#             # Ensure the query is mapped to the new corpus ID
#             if query_id not in new_qrels:
#                 new_qrels[query_id] = {}  # Initialize query entry if missing
            
#             new_qrels[query_id][new_corpus_id] = 1  # Assign relevance score

# corpus.update(new_corpus)
# qrels.update(new_qrels)
# tasks['cosqa'] = (corpus, queries, qrels)

# print(f"Total docs in expanded corpus: {len(corpus)}")
# print(f"Total qrels entries: {sum(len(v) for v in qrels.values())}")

# # Print first few new corpus and qrel entries
# print(next(iter(new_corpus.items())))  # Print first new corpus entry
# print(next(iter(new_qrels.items())))  # Print first new qrel entry


In [10]:
# corpus, queries, qrels = tasks['cosqa']
# corpus_ids = sorted(corpus, key=lambda k: len(corpus[k].get("title", "") + corpus[k].get("text", "")), reverse=True)
# corpus = [corpus[cid] for cid in corpus_ids]

In [11]:
# all_texts = ["passage: "+ doc['text'] for doc in corpus]
# len(all_texts)

In [12]:
# qrels['q20105']

In [13]:
# for doc in corpus.values():
#     print(doc)
#     break

#### single explanation

In [14]:
# # Unpack the tuple
corpus, queries, qrels = tasks['cosqa']

for _, row in expl_df.iterrows():
    corpus_id = row['corpus-id']
    explanation = row['explanation_deepseek_1_cleaned']

    if corpus_id in corpus and explanation and explanation.strip():
        corpus[corpus_id]['text'] = explanation  

corpus = {doc_id: doc for doc_id, doc in corpus.items() if 'text' in doc and doc['text'].strip()}
print(corpus['d1'])
tasks['cosqa'] = (corpus, queries, qrels)

print(f"Total docs in corpus after replacement: {len(tasks['cosqa'][0])}")
missing_text = sum(1 for doc in tasks['cosqa'][0].values() if not doc.get('text'))
print(f"Number of docs still missing 'text': {missing_text}")


{'text': 'The doc string and code both write the boolean value 1 to the stream. The code uses a condition based on n, flipping between TRUE and FALSE values. The stream is a mechanism for writing to a binary stream.The docstring and code both write the boolean value 1 to the stream. The code uses a condition based on n, flipping between TRUE and FALSE values. The stream is a mechanism for writing to a binary stream.', 'title': ''}
Total docs in corpus after replacement: 20604
Number of docs still missing 'text': 0


In [15]:
print(corpus['d20604'])

{'text': 'The code snippet defines a function `_unzip_handle` that takes a `handle` as input. If the handle is a string, it uses `_gzip_open_filename` to convert it, otherwise, it uses `_gzip_open_handle`. The function returns the handle after conversion. The docstring explains that it unzips the file handle, handling strings by converting them to filename handles. The code is concise and explains the process clearly.', 'title': ''}


### Running the pipeline

In [16]:
model_name = "intfloat/e5-base-v2"
model = YourCustomDEModel(model_name=model_name)

evaluation = COIR(tasks=tasks,batch_size=256)

results = evaluation.run(model, output_folder=f"results/{model_name}/explanation_deepseek_1_baseline1")
print(results)

YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

len of queries:  500


Encoding batches: 100%|██████████| 2/2 [00:01<00:00,  1.87batch/s]


len of corpus:  20604


Encoding batches: 100%|██████████| 81/81 [00:32<00:00,  2.51batch/s]


in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

Retrieval evaluation results saved to results/intfloat/e5-base-v2/explanation_deepseek_1_baseline1
{'cosqa': {'NDCG': {'NDCG@1': 0.218, 'NDCG@3': 0.29717, 'NDCG@5': 0.32857, 'NDCG@10': 0.37425, 'NDCG@100': 0.43706, 'NDCG@1000': 0.4518}, 'MAP': {'MAP@1': 0.218, 'MAP@3': 0.27633, 'MAP@5': 0.29333, 'MAP@10': 0.31243, 'MAP@100': 0.3251, 'MAP@1000': 0.32573}, 'Recall': {'Recall@1': 0.218, 'Recall@3': 0.358, 'Recall@5': 0.436, 'Recall@10': 0.576, 'Recall@100': 0.874, 'Recall@1000': 0.986}, 'Precision': {'P@1': 0.218, 'P@3': 0.11933, 'P@5': 0.0872, 'P@10': 0.0576, 'P@100': 0.00874, 'P@1000': 0.00099}}}


In [17]:
# models_list = ["BAAI/bge-base-en", "intfloat/e5-base-v2", "sentence-transformers/all-MiniLM-L6-v2", "microsoft/unixcoder-base", "infly/inf-retriever-v1-1.5b", "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "google-bert/bert-base-uncased"]  # Add your models here

# for model_name in models_list:
#     model = YourCustomDEModel(model_name=model_name)
    
#     evaluation = COIR(tasks=tasks, batch_size=128)
    
#     results = evaluation.run(model, output_folder=f"results/{model_name}_explanation_granite_1")
    
#     print(f"Results for {model_name}: {results}")
